# Option 1. Initialize without telemetry

* For first cluster

In [22]:
%%bash
terraform output -raw cluster1_init_remote
eval "$(terraform output -raw cluster1_init_remote)"

  # Initialize Vault
export VAULT_ADDR=https://vault-eu-primary-europe-west1-cqnb.josemerchan-8d4c7e.gcp.sbx.hashicorpdemo.com:8200
export SKIP_TLS_VERIFY=true
vault operator init -format=json > output.json
cat output.json | jq -r .root_token > root.token
export VAULT_TOKEN=$(cat root.token)
sleep 10

# Save info in GCP Secrets
gcloud secrets create root_token_europe-west1_cqnb --replication-policy="automatic" --project=hc-481920a3f7e54d39b33d0454ff9
echo -n $VAULT_TOKEN | gcloud secrets versions add root_token_europe-west1_cqnb --project=hc-481920a3f7e54d39b33d0454ff9 --data-file=-
gcloud secrets create vault_init_data_europe-west1_cqnb --replication-policy="automatic" --project=hc-481920a3f7e54d39b33d0454ff9
cat output.json | gcloud secrets versions add vault_init_data_europe-west1_cqnb --project=hc-481920a3f7e54d39b33d0454ff9 --data-file=-
rm output.json
rm root.token

# Enable Audit Logging
vault audit enable file file_path=/var/log/vault.log

# Enable Dead Server clean-up, min-quo

Created secret [root_token_europe-west1_cqnb].
Created version [1] of the secret [root_token_europe-west1_cqnb].
Created secret [vault_init_data_europe-west1_cqnb].
Created version [1] of the secret [vault_init_data_europe-west1_cqnb].


Success! Enabled the file audit device at: file/


created key [10a18e5f87bae8fc750ed27b5db78b6d6064898c] of type [json] as [sa-keys__europe-west1_cqnb.json] for [iam-vault-cqnb@hc-481920a3f7e54d39b33d0454ff9.iam.gserviceaccount.com]


Success! Data written to: sys/storage/raft/snapshot-auto/config/hourly


* For second cluster

In [23]:
%%bash
terraform output -raw cluster2_init_remote
eval "$(terraform output -raw cluster2_init_remote)"

  # Initialize Vault
export VAULT_ADDR=https://vault-eu-secondary-europe-west1-iovc.josemerchan-8d4c7e.gcp.sbx.hashicorpdemo.com:8200
export SKIP_TLS_VERIFY=true
vault operator init -format=json > output.json
cat output.json | jq -r .root_token > root.token
export VAULT_TOKEN=$(cat root.token)
sleep 10

# Save info in GCP Secrets
gcloud secrets create root_token_europe-west1_iovc --replication-policy="automatic" --project=hc-481920a3f7e54d39b33d0454ff9
echo -n $VAULT_TOKEN | gcloud secrets versions add root_token_europe-west1_iovc --project=hc-481920a3f7e54d39b33d0454ff9 --data-file=-
gcloud secrets create vault_init_data_europe-west1_iovc --replication-policy="automatic" --project=hc-481920a3f7e54d39b33d0454ff9
cat output.json | gcloud secrets versions add vault_init_data_europe-west1_iovc --project=hc-481920a3f7e54d39b33d0454ff9 --data-file=-
rm output.json
rm root.token

# Enable Audit Logging
vault audit enable file file_path=/var/log/vault.log

# Enable Dead Server clean-up, min-q

Error initializing: Put "https://vault-eu-secondary-europe-west1-iovc.josemerchan-8d4c7e.gcp.sbx.hashicorpdemo.com:8200/v1/sys/init": unexpected EOF
Created secret [root_token_europe-west1_iovc].
ERROR: (gcloud.secrets.versions.add) INVALID_ARGUMENT: Secret Payload cannot be empty.
Created secret [vault_init_data_europe-west1_iovc].
ERROR: (gcloud.secrets.versions.add) INVALID_ARGUMENT: Secret Payload cannot be empty.
Error enabling audit device: Error making API request.

URL: PUT https://vault-eu-secondary-europe-west1-iovc.josemerchan-8d4c7e.gcp.sbx.hashicorpdemo.com:8200/v1/sys/audit/file
Code: 403. Errors:

* 2 errors occurred:
	* permission denied
	* invalid token


Error making API request.

URL: PUT https://vault-eu-secondary-europe-west1-iovc.josemerchan-8d4c7e.gcp.sbx.hashicorpdemo.com:8200/v1/sys/storage/raft/autopilot/configuration
Code: 403. Errors:

* 2 errors occurred:
	* permission denied
	* invalid token


created key [df0b7bf8c341bf4140a55aad51504cefd7ff0b05] of type 

CalledProcessError: Command 'b'terraform output -raw cluster2_init_remote\neval "$(terraform output -raw cluster2_init_remote)"\n'' returned non-zero exit status 2.

# Option 2. Initialize with telemetry

* Initialize the first cluster. Requires SSH access via console.
* Enable Audit logging
* Obtain a token and configure the agent to use it

In [7]:
%%bash
terraform output -raw cluster1_init_script_node1


# Initialize Vault
vault status # wait for vault to show as unsealed

vault operator init -format=json > output.json
cat output.json | jq -r .root_token > root.token
export VAULT_TOKEN=$(cat root.token)

# Save info in GCP Secrets
gcloud secrets create root_token_europe-west1_1toq --replication-policy="automatic"
echo -n $VAULT_TOKEN | gcloud secrets versions add root_token_europe-west1_1toq --data-file=-
gcloud secrets create vault_init_data_europe-west1_1toq --replication-policy="automatic"
cat output.json | gcloud secrets versions add vault_init_data_europe-west1_1toq --data-file=-

# Enable Audit Logging
vault audit enable file file_path=/var/log/vault.log

# Create Prometheus token and update Ops Agent Configuration with Token
vault policy write prometheus-metrics /opt/vault/prometheus.hcl
vault token create -field=token -policy prometheus-metrics > prometheus.token
export PROMETHEUS_TOKEN=$(cat prometheus.token)
sudo sed -ibak "s/VAULT_TOKEN_PROMETHEUS/$PROMETHEUS_TOKEN/g" /etc/

* Obtain token for Ops Agent
* Restart Ops Agent Service
* Repeat on nodes 2 to X

In [8]:
%%bash
terraform output -raw cluster1_init_script_node2-X


# Obtain root token
export VAULT_TOKEN=$(gcloud secrets versions access latest --secret=root_token_europe-west1_1toq)

# Create Prometheus token and update Ops Agent Configuration with Token
vault token create -field=token -policy prometheus-metrics > prometheus.token
export PROMETHEUS_TOKEN=$(cat prometheus.token)
sudo sed -ibak "s/VAULT_TOKEN_PROMETHEUS/$PROMETHEUS_TOKEN/g" /etc/google-cloud-ops-agent/config.yaml
sudo systemctl restart google-cloud-ops-agent



* Enable Automatic Backups on local terminal

In [11]:
%%bash
terraform output -raw cluster1_init_auto_snapshot


# From your terminal
# Enable automatic snapshots
export VAULT_TOKEN=$(gcloud secrets versions access latest --secret=root_token_europe-west1_1toq --project=hc-481920a3f7e54d39b33d0454ff9)
gcloud iam service-accounts keys create sa-keys__europe-west1_1toq.json --iam-account=iam-vault-1toq@hc-481920a3f7e54d39b33d0454ff9.iam.gserviceaccount.com
VAULT_ADDR=https://vault-eu-primary-europe-west1-1toq.josemerchan-8d4c7e.gcp.sbx.hashicorpdemo.com:8200 vault write sys/storage/raft/snapshot-auto/config/hourly interval="1h" retain=10 path_prefix="snapshots/" storage_type=google-gcs google_gcs_bucket=gcs-vault-snapshot-1toq google_service_account_key="@sa-keys__europe-west1_1toq.json"



* Connect to cluster

In [17]:
%%bash
terraform output -raw cluster1_fqdn_8200

https://vault-eu-primary-europe-west1-1toq.josemerchan-8d4c7e.gcp.sbx.hashicorpdemo.com:8200

* Repeat for other clusters

In [12]:
%%bash
terraform output -raw cluster2_init_script_node1


# Initialize Vault
vault status # wait for vault to show as unsealed

vault operator init -format=json > output.json
cat output.json | jq -r .root_token > root.token
export VAULT_TOKEN=$(cat root.token)

# Save info in GCP Secrets
gcloud secrets create root_token_europe-west1_ndcj --replication-policy="automatic"
echo -n $VAULT_TOKEN | gcloud secrets versions add root_token_europe-west1_ndcj --data-file=-
gcloud secrets create vault_init_data_europe-west1_ndcj --replication-policy="automatic"
cat output.json | gcloud secrets versions add vault_init_data_europe-west1_ndcj --data-file=-

# Enable Audit Logging
vault audit enable file file_path=/var/log/vault.log

# Create Prometheus token and update Ops Agent Configuration with Token
vault policy write prometheus-metrics /opt/vault/prometheus.hcl
vault token create -field=token -policy prometheus-metrics > prometheus.token
export PROMETHEUS_TOKEN=$(cat prometheus.token)
sudo sed -ibak "s/VAULT_TOKEN_PROMETHEUS/$PROMETHEUS_TOKEN/g" /etc/

In [13]:
%%bash
terraform output -raw cluster2_init_script_node2-X


# Obtain root token
export VAULT_TOKEN=$(gcloud secrets versions access latest --secret=root_token_europe-west1_ndcj)

# Create Prometheus token and update Ops Agent Configuration with Token
vault token create -field=token -policy prometheus-metrics > prometheus.token
export PROMETHEUS_TOKEN=$(cat prometheus.token)
sudo sed -ibak "s/VAULT_TOKEN_PROMETHEUS/$PROMETHEUS_TOKEN/g" /etc/google-cloud-ops-agent/config.yaml
sudo systemctl restart google-cloud-ops-agent



In [14]:
%%bash
terraform output -raw cluster2_init_auto_snapshot


# From your terminal
# Enable automatic snapshots
export VAULT_TOKEN=$(gcloud secrets versions access latest --secret=root_token_europe-west1_ndcj --project=hc-481920a3f7e54d39b33d0454ff9)
gcloud iam service-accounts keys create sa-keys__europe-west1_ndcj.json --iam-account=iam-vault-ndcj@hc-481920a3f7e54d39b33d0454ff9.iam.gserviceaccount.com
VAULT_ADDR=https://vault-eu-secondary-europe-west1-ndcj.josemerchan-8d4c7e.gcp.sbx.hashicorpdemo.com:8200 vault write sys/storage/raft/snapshot-auto/config/hourly interval="1h" retain=10 path_prefix="snapshots/" storage_type=google-gcs google_gcs_bucket=gcs-vault-snapshot-ndcj google_service_account_key="@sa-keys__europe-west1_ndcj.json"



* Connect to cluster

In [18]:
%%bash
terraform output -raw cluster2_fqdn_8200

https://vault-eu-secondary-europe-west1-ndcj.josemerchan-8d4c7e.gcp.sbx.hashicorpdemo.com:8200